In [1]:
import os
from multiprocessing import Pool
import pandas as pd
import numpy as np
from tqdm import tnrange
from sklearn.preprocessing import LabelEncoder

In [2]:
%%time
big_data=pd.read_hdf('raw_data/big.hdf',key='train')

CPU times: user 152 ms, sys: 11.3 s, total: 11.5 s
Wall time: 2min 5s


In [3]:
int_fea_num=13
cat_fea_num=26

cat_uniqs=[(c,len(big_data[c].unique())) for c in range(int_fea_num,int_fea_num+cat_fea_num)]
cat_uniqs=sorted(cat_uniqs,key=lambda x:x[1],reverse=True)

In [7]:
big_data[:int_fea_num].head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,1.0,...,9,5447,274,2,235210,17,3,234192,88,87661
1,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,1.0,...,0,3873,274,0,2959992,17,3,79957,88,66604
2,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,1.0,...,6,1118,2177,3,7013400,11,3,69414,104,148164
3,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,NaN,...,1,2550,2177,3,3273973,17,3,171362,104,148164
4,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,1.0,...,1,838,2177,3,1031384,17,2,212017,104,148164


In [11]:
type(big_data.iat[3,0])

numpy.float64

In [12]:
np.isnan(big_data.iat[3,0])

True

In [13]:
big_data.reset_index(inplace=True)
big_data[-5:]

,index,0,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
51882747,6042130,NaN,2,6.0,9.0,20463.0,NaN,0.0,14.0,40.0,...,8,421,274,2,3285412,17,2,90116,1,87879
51882748,6042131,1.0,44,1.0,NaN,135.0,8.0,1.0,0.0,0.0,...,8,817,274,1,6519536,11,3,42556,0,120286
51882749,6042132,NaN,1,6.0,5.0,283.0,26.0,81.0,5.0,42.0,...,9,1237,274,0,2135976,17,2,197311,0,27759
51882750,6042133,0.0,308,25.0,5.0,36326.0,NaN,0.0,4.0,6.0,...,0,3367,274,0,7282674,17,2,253463,91,103627
51882751,6042134,0.0,0,6.0,NaN,2976.0,10.0,2.0,0.0,221.0,...,4,3367,274,0,3807486,17,2,122858,91,88320


In [28]:
big_data.drop('index',axis=1,inplace=True)
big_data.iat[0,21]

2

In [43]:
# (True & (big_data[15]==big_data.iat[0,15]) & (big_data[16]==big_data.iat[0,16])).sum()
big_data[(True & (big_data[15]==big_data.iat[0,15]) & (big_data[16]==big_data.iat[0,16]))][11].count()

0

In [49]:
cat_uniqs=sorted(cat_uniqs,key=lambda x:x[1])

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [66]:
cat_uniqs[:5]

[(21, 3), (32, 4), (29, 10), (35, 15), (34, 18)]

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [71]:
(big_data[15]>-1).sum()

51882752

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [73]:
(big_data[cat_uniqs[0][0]]==big_data.iat[0,cat_uniqs[0][0]]).sum()

46684518

In [80]:
cond_set={}
def get_cond(i,col,val):
    if (col,val) not in cond_set:
        cond=big_data[col]==val
        cond_set[(col,val)]=cond
    return cond_set[(col,val)]


            

HBox(children=(IntProgress(value=0, max=51882752), HTML(value='')))

process 3 2


/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


cond sum 20411114
cond sum 871196
process 4 11
cond sum 20411114
cond sum 871196
process 5 2
cond sum 20411114
cond sum 1317784
process 6 3
cond sum 20411114


KeyboardInterrupt: 

In [81]:
for i in tnrange(big_data.shape[0]):
    for c in range(int_fea_num):
        if np.isnan(big_data.iat[i,c]):
            print('process %d %d'%(i,c))
            set_v=False
            cond=get_cond(i,cat_uniqs[0][0],big_data.iat[i,cat_uniqs[0][0]])
            last_series=big_data[cond][c]
            
            for j in range(1,cat_fea_num):
                cond = (cond) & (get_cond(i,cat_uniqs[j][0],big_data.iat[i,cat_uniqs[j][0]]))
                print('cond sum %d'%cond.sum())
                series=big_data[cond][c]
                if series.count() < 10000000:
                    cond_sum=series.count()
                    big_data.iat[i,c]=series.quantile()
                    set_v=True
                    break
                else:
                    last_series=series
                    
            if set_v:
                break
            else:
                print('not set %d %c'%(i,c))

HBox(children=(IntProgress(value=0, max=51882752), HTML(value='')))

process 3 3


/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


cond sum 20411114
cond sum 871196
process 5 3
cond sum 20411114
cond sum 1317784
process 6 3
cond sum 20411114
cond sum 1317784
process 8 9
cond sum 20411114
cond sum 9504512
process 9 2
cond sum 20411114
cond sum 2356884
process 10 0
cond sum 980411
process 11 11
cond sum 20411114
process 12 2
cond sum 20411114
cond sum 9504512
process 13 0
cond sum 8759668
process 14 11
cond sum 8837535
process 15 0
cond sum 20411114
cond sum 1317784
process 16 2
cond sum 8837535
process 17 2
cond sum 20411114
cond sum 9504512
process 18 11
cond sum 8837535
process 19 2
cond sum 20411114
cond sum 9504512
process 20 0
cond sum 20411114
cond sum 9504512
process 21 11
cond sum 8759668
process 22 0
cond sum 20411114
cond sum 2678233
process 23 0
cond sum 20411114
cond sum 871196
process 24 3
cond sum 20411114
cond sum 2356884
process 25 0
cond sum 8759668
process 28 11
cond sum 20411114
process 29 11
cond sum 20411114
process 30 0
cond sum 20411114
cond sum 9504512
process 31 0
cond sum 20411114
cond sum

KeyboardInterrupt: 